# Example for calculating matrix Green's function derivative using Loewner theorem
Tsung-Han Lee

In [1]:
"""
Test for Loewner matrix of a Green's function type, here I is identity matrix and A is an off-diagonal matrix
G = [wI+A]^(-1)
"""
import sys
import os
import numpy as np
from scipy.linalg import sqrtm, inv, block_diag, eigh
from scipy.optimize import brentq, bisect, newton
from math import *
from sympy import *
np.set_printoptions(precision=4, suppress=True)

Here we choose a simple form of matrix Green's function
$$G(\omega) = \frac{1}{\omega I + A}$$,
where
$$A=\begin{bmatrix} 0 & a \\ a & 0  \\ \end{bmatrix}, $$
and 
$$I=\begin{bmatrix} 1 & 0 \\ 0 & 1  \\ \end{bmatrix}. $$

We can analytically calculate the inversion by Cramer's rule
$$G(\omega) = \frac{\begin{bmatrix} \omega & -a \\ -a & \omega  \\ \end{bmatrix}}{\omega^2 - a^2}$$
Therefore, the object we want to calculate can be analytical calculated, which is
$$\frac{d\frac{1}{G(\omega)_{0,0}}}{d\omega}|_{\omega=\omega^*}=\frac{\omega^{*2}+a^2}{\omega^{*2}}$$

In [30]:
"""
Analytic solution
Ginv = [[w,a],[a,w]]
G = [[w, -a],[-a, w]]/(w**2 -a**2)
d(1/G[0,0])/dw = (w**2+a**2)/w**2, diverge at w->0.
here we evaluate derivative at w=ws and ws=1, a=2. 
"""
ws=0.1; a=2.
test = inv(np.array([[ws,a],[a,ws]]))
assert np.allclose(test, np.array([[ws,-a],[-a,ws]])/(ws**2-a**2))
#Since it diverge we can evaluate it at the other position ws.
print "analytic derivative=",(ws**2+a**2)/ws**2

analytic derivative= 401.0


In order to use the Loewner matrix theorem,
$$Df(A)(H)=\frac{d}{dt}(A+tH)|_{t=0}=f^{[1]}(A) \circ H.$$
We need to shift the $\omega$ to $\omega^*$.
Therefore, the matrix Green's function become
$$G(\omega) = \frac{1}{(\omega - \omega^* )I + A },$$
now, $A=\begin{bmatrix} \omega^* & a \\ a & \omega^*  \\ \end{bmatrix}$.

We want to calculate the derivative
$$\frac{d\frac{1}{G(\omega)_{0,0}}}{d\omega}|_{\omega=\omega^*}=\frac{d}{d(\omega-\omega^*)}
\frac{1}{([(\omega - \omega^* )I + A ]^{-1})_{0,0}}|_{\omega-\omega^*=0} \\
=\frac{-1}{G(\omega^*)^2_{0,0}}[\frac{d}{d(\omega-\omega^*)}[(\omega-\omega^*)I+A]^{-1}|_{\omega-\omega^*=0}]_{0,0}$$


We, then, can use the Loewner theorem to evaluate the derivative at $\omega-\omega^*=0$ by letting $H=I$ and $\omega-\omega^*=t$. The calculation of Loewner matrix, $f^{[1]}(A)$, and the Schur product, $f^{[1]}(A) \circ H$, should be evaluate at the basis diagonalizing $A$, then transform back to the original basis. The Loewner matrix can be found in the book, "Positive Definite Matrices" p.165 by Bhatia.

In [32]:
"""
Calculate the derivative at ws using Loewner matrix, we need to shift wp=w-ws, since
Loewner matrix needs to be evaluate at wp=0. Here we set  a=2. Now we have 
Ginv = (wp*I + [[ws,a],[a,ws]])
"""
#ws = 3.; a=2. # evaluate derivative at ws
H = np.eye(2,2)
A = np.array([[ws,a],[a,ws]])
onebyg00sqr = 1./(inv(A)[0,0])**2 #1/G(ws)_{0,0}^2
evals, evecs = eigh(A)
Hbar = np.dot(np.conj(evecs).T, np.dot( H, evecs) ) #transform H to A's basis
#print Hbar

#create Loewner matrix in A's basis
loewm = np.zeros(evecs.shape)
for i in range(loewm.shape[0]):
    for j in range(loewm.shape[1]):
        if i==j:
            #print -1./(evals[i])**2
            loewm[i,i] = -1./(evals[i])**2
        if i!=j:
            #print ( (1./evals[i]) - (1./evals[j]) )/(evals[i]-evals[j])
            loewm[i,j]= ( (1./evals[i]) - (1./evals[j]) )/(evals[i]-evals[j])

#Perform the Schur product in A's basis then transform back to original basis.
deriv = -1.*onebyg00sqr*np.dot(evecs, np.dot( loewm*Hbar, np.conj(evecs).T ) )[0,0]
print "Loewner theorem derivative=",deriv
print "analytic derivative=",(ws**2+a**2)/ws**2

Loewner theorem derivative= 401.0
analytic derivative= 401.0
